# Assignment 2:
The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [48]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.

# * Make sure you have already installed tensorflow.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in 1_notmnist.ipynb

In [49]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:

+ data as a flat matrix,

+ labels as float 1-hot encodings.

In [50]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:

+ First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

        with graph.as_default():
            ...
            
+ Then you can run the operations on this graph as many times as you want by calling session.run(), providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:
        with tf.Session(graph=graph) as session:
            ...

#### Load data and build a graph
Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [51]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

####
# INPUT DATA
####
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

####
# VARIABLES
####
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))

####
# Training computation.
####
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

####
# Optimizer.
####
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

####  
# Predictions for the training, validation, and test data.
####
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

#### Compilation and Iteration
Let's run this computation and iterate:

In [52]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 17.755375
Training accuracy: 14.6%
Validation accuracy: 16.2%
Loss at step 100: 2.320363
Training accuracy: 72.2%
Validation accuracy: 71.2%
Loss at step 200: 1.868323
Training accuracy: 74.7%
Validation accuracy: 73.1%
Loss at step 300: 1.621270
Training accuracy: 76.0%
Validation accuracy: 73.6%
Loss at step 400: 1.455895
Training accuracy: 77.0%
Validation accuracy: 74.1%
Loss at step 500: 1.334109
Training accuracy: 77.7%
Validation accuracy: 74.3%
Loss at step 600: 1.239164
Training accuracy: 78.2%
Validation accuracy: 74.5%
Loss at step 700: 1.162184
Training accuracy: 78.7%
Validation accuracy: 74.6%
Loss at step 800: 1.097961
Training accuracy: 79.2%
Validation accuracy: 74.9%
Test accuracy: 82.5%


#### Stochastic Gradient Descent
Let's now switch to stochastic gradient descent training instead, which is much faster.
The graph will be similar, except that instead of holding all the training data into a constant node, we create a Placeholder node which will be fed actual data at every call of session.run().

Placeholders will be put to tensorflow for running the computation. The format is 2D tensor floating number with the batch size and image shape, otherwise, a dimension can be of any length.

In [53]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

#### Computation and iteration
Let's run it:

In [54]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    ####
    # offset is explained right below.
    ####
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 14.545450
Minibatch accuracy: 19.5%
Validation accuracy: 21.9%
Minibatch loss at step 500: 2.003801
Minibatch accuracy: 68.0%
Validation accuracy: 75.1%
Minibatch loss at step 1000: 1.195774
Minibatch accuracy: 78.1%
Validation accuracy: 76.6%
Minibatch loss at step 1500: 0.838895
Minibatch accuracy: 83.6%
Validation accuracy: 77.7%
Minibatch loss at step 2000: 0.973434
Minibatch accuracy: 76.6%
Validation accuracy: 77.8%
Minibatch loss at step 2500: 0.847659
Minibatch accuracy: 79.7%
Validation accuracy: 77.3%
Minibatch loss at step 3000: 1.028889
Minibatch accuracy: 76.6%
Validation accuracy: 78.1%
Test accuracy: 85.6%


#### Offset explanation
* The offset offers an arithmetic sequence within each epoch. 
* Different offsets can be obtained from different epochs.
* Each epoch is the measure of #times all the train_datasets are used once to update the weights.
* For batch training, all the training samples pass through the learning algorithm simultaneously in one epoch before the weights are updated.
* For sequential training, all the weights are updated after each training vector is sequentially passed through the training algorithm.
* The offsets make each mini-batch different from each other within each epoch, and among them.
Example: if batch_size=3, train_labels=100:
+ steps=1->32; offset=3,6,..,96;
+ steps=33->64; offset=2,5,..,95;
+ steps=65->96; offset=1,4,..,94;

# Problem:
    Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units nn.relu() and 1024 hidden nodes. This model should improve your validation / test accuracy.
    
    X -> *W +b -> ReLUs (H:1024) -> y -> softmax(y) -> probabilities.

In [85]:
#import tensorflow as tf
# parameters
batch_size = 128
image_size = 28

# network params
num_labels = 10 # total classes A -> J
num_hidden = 1024 # number of hidden nodes

# create graph
graph = tf.Graph()
with graph.as_default():
    # tensorflow graph input
    # At run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden]))
    biases_1 = tf.Variable(tf.zeros([num_hidden]))
    
    weights_2 = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1 ## 
    # ReLUs
    relu_layer = tf.nn.relu(logits_1)
    
    logits = tf.matmul(relu_layer, weights_2) + biases_2
    # loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training
    train_prediction = tf.nn.softmax(logits)
    
    # Predictions for validation 
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_valid = tf.matmul(relu_layer, weights_2) + biases_2
    
    valid_prediction = tf.nn.softmax(logits_valid)
    
    # Predictions for test data
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_test = tf.matmul(relu_layer, weights_2) + biases_2
    
    test_prediction = tf.nn.softmax(logits_test)


Let's run it!

In [89]:
num_steps = 3001

with tf.Session(graph=graph) as session:
#   tf.initialize_all_variables().run()
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 252.399597
Minibatch accuracy: 14.8%
Validation accuracy: 27.8%
Minibatch loss at step 500: 18.908394
Minibatch accuracy: 71.1%
Validation accuracy: 80.2%
Minibatch loss at step 1000: 8.688184
Minibatch accuracy: 82.8%
Validation accuracy: 80.8%
Minibatch loss at step 1500: 2.709536
Minibatch accuracy: 84.4%
Validation accuracy: 81.0%
Minibatch loss at step 2000: 5.841818
Minibatch accuracy: 82.0%
Validation accuracy: 81.1%
Minibatch loss at step 2500: 2.895524
Minibatch accuracy: 82.0%
Validation accuracy: 81.6%
Minibatch loss at step 3000: 6.621354
Minibatch accuracy: 82.0%
Validation accuracy: 81.2%
Test accuracy: 88.8%


#### End of assignment 2: A fully connected network using stochastic gradient descent and ReLUs with tensorflow.
The new model gives better test accuracy: 88.8%, compared to the model without ReLUs activation (85.6%), as adding one or more hidden layers to the network allows it to model more complex functions.